In [2]:

import os
import random
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from IPython.display import HTML


import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.optim import Adam
import torchvision.datasets as dsets


import torchgan
from torchgan.models import *
from torchgan.losses import *
from torchgan.trainer import Trainer
import pandas as pd
import torch
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
from torchgan.models import DCGANGenerator, DCGANDiscriminator
from torchgan.losses import LeastSquaresGeneratorLoss, LeastSquaresDiscriminatorLoss
from torchgan.trainer import Trainer
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset


In [3]:
class NFUQNIDS(Dataset):
    def __init__(self, data_file, transform=None, target_transform=None):
        self.data = pd.read_csv(data_file)
        self.labels = self.data['Label']
        self.data = self.data.drop({'Label', 'Attack', 'Dataset'}, axis=1)
        self.data = self.data.values

        # Standardize the features
        scaler = StandardScaler()
        self.data = scaler.fit_transform(self.data)

        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = self.data[idx]
        label = self.labels.iloc[idx]

        if self.transform:
            item = self.transform(item)

        if self.target_transform:
            label = self.target_transform(label)

        # Return data in the format expected by TorchGAN
        return {'data': torch.tensor(item, dtype=torch.float32),
                'label': torch.tensor(label, dtype=torch.float32)}

In [4]:

data_file = 'data/cleaned/NF-UQ-NIDS-CLEANED.csv'
epochs = 10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

transform = None  

In [5]:

nfu_dataset = NFUQNIDS(data_file=data_file, transform=transform)


In [6]:

dcgan_network = {
    "generator": {
        "name": DCGANGenerator,
        "args": {
            "encoding_dims": 100,
            "out_channels": 1,
            "step_channels": 32,
            "nonlinearity": torch.nn.LeakyReLU(0.2),
            "last_nonlinearity": torch.nn.Tanh(),
        },
        "optimizer": {"name": torch.optim.Adam, "args": {"lr": 0.0001, "betas": (0.5, 0.999)}},
    },
    "discriminator": {
        "name": DCGANDiscriminator,
        "args": {
            "in_channels": 1,
            "step_channels": 32,
            "nonlinearity": torch.nn.LeakyReLU(0.2),
            "last_nonlinearity": torch.nn.LeakyReLU(0.2),
        },
        "optimizer": {"name": torch.optim.Adam, "args": {"lr": 0.0003, "betas": (0.5, 0.999)}},
    },
}

In [7]:

lsgan_losses = [LeastSquaresGeneratorLoss(), LeastSquaresDiscriminatorLoss()]


In [8]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.backends.cudnn.deterministic = True
    epochs = 10
else:
    device = torch.device("cpu")
    epochs = 5

print("Device: {}".format(device))
print("Epochs: {}".format(epochs))

Device: cuda:0
Epochs: 10


In [9]:

batch_size = 64 
dataloader = data.DataLoader(nfu_dataset, batch_size=batch_size, shuffle=True)

# Trainer initialization
trainer = Trainer(dcgan_network, lsgan_losses, sample_size=batch_size, epochs=epochs, device=device)

# Training loop
for epoch in range(epochs):
    for real_batch in dataloader:
        # Training the GAN
        trainer.train(real_batch)


    trainer.end_epoch_hook()


noise = torch.randn(batch_size, 100).to(device) 
fake_data = dcgan_network['generator'](noise)


plt.figure(figsize=(10, 5))
plt.title("Fake Data")
plt.plot(fake_data[0].detach().cpu().numpy())  
plt.show()


AttributeError: 'str' object has no attribute 'size'